In [1]:
from qiskit import *
import numpy as np
from qiskit.circuit.library import MCXGate
from qiskit.primitives import *

Because my code can only encode the value that is integer, We have to change the value 

And also our lattice size is 2^n * 2^n, so it is needed to add padding.

In [11]:
input_value = np.array([[0,0.5,0],[0.8,1,0.8],[0,0.5,0]])
modif_value = input_value*10
modif_value = list(modif_value)
for index,i in enumerate(modif_value):
    while(len(i))<4:
        i = list(i)
        i.append(0.)
    modif_value[index] = i
    
modif_value.append([0.]*4)
modif_value = np.array(modif_value,dtype=int)
print(modif_value)

[[ 0  5  0  0]
 [ 8 10  8  0]
 [ 0  5  0  0]
 [ 0  0  0  0]]


In [14]:
def data_in_bit(inputs):
    res = []
    for i in inputs:
        temp = []
        for j in i:
            binary = format(j,'b').zfill(4)
            binary = reverse_str(binary)
            temp.append(binary)
        res.append(temp)
    return np.array(res)
def reverse_str(str_):
    res = ''
    for i in range(len(str_)):
        res = res + str_[-i-1]
    return res

In [15]:
## Change the data into binary form
binary_inputs = data_in_bit(modif_value)
print(binary_inputs)

[['0000' '1010' '0000' '0000']
 ['0001' '0101' '0001' '0000']
 ['0000' '1010' '0000' '0000']
 ['0000' '0000' '0000' '0000']]


In [16]:
X_index = QuantumRegister(2,name='X_layer')
Y_index = QuantumRegister(2,name='Y_layer')
coeffs = QuantumRegister(4,name='coeffs_layer')

qc = QuantumCircuit(X_index,Y_index,coeffs)
qc.draw('mpl')

for i in range(2):
    qc.h(X_index[i])
for i in range(2):
    qc.h(Y_index[i])

for i in range(4):
    for j in range(4):
        X_idx = format(i,'b').zfill(2)
        Y_idx = format(j,'b').zfill(2)
        coeff = binary_inputs[i][j]
        gate = MCXGate(4,ctrl_state=X_idx+Y_idx)
        for index,k in enumerate(coeff):
            if k == '1':
                qc.append(gate,X_index[:]+Y_index[:]+[coeffs[index]])
        qc.barrier()
qc.measure_all()

In [17]:
sampler = Sampler()

result = sampler.run(qc).result().quasi_dists[0]
binary_dict = {}

for i in result.keys():
    binary_dict[format(i,'b').zfill(8)] = result[i]

print(binary_dict)

{'00000000': 0.0625, '00000010': 0.0625, '00000011': 0.0625, '00000111': 0.0625, '00001000': 0.0625, '00001010': 0.0625, '00001011': 0.0625, '00001100': 0.0625, '00001101': 0.0625, '00001110': 0.0625, '00001111': 0.0625, '01010001': 0.0625, '01011001': 0.0625, '10000100': 0.0625, '10000110': 0.0625, '10100101': 0.0625}


In [22]:
## From this result we can re construct the input
decoded_result = []
for i in range(4):
    temp = []
    for j in range(4):
        X_idx = format(i,'b').zfill(2)
        Y_idx = format(j,'b').zfill(2)
        key = X_idx+Y_idx
        
        for k in binary_dict.keys():
            if k[4:] == key:
                temp.append(int(k[:4],2))
    decoded_result.append(temp)        
decoded_result = np.array(decoded_result)/10
print('Decoded result :\n',decoded_result) 
print('\n')
print('initial input :\n', input_value)
print('\n')
print('The result is same')

Decoded result :
 [[0.  0.5 0.  0. ]
 [0.8 1.  0.8 0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.  0. ]]


initial input :
 [[0.  0.5 0. ]
 [0.8 1.  0.8]
 [0.  0.5 0. ]]


The result is same
